In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from deep_convnet import DeepConvNet
%matplotlib inline

Using TensorFlow backend.


In [2]:
# データの読み込み
(x_train, t_train), (x_test, t_test) = mnist.load_data()

# 正規化
x_test = x_test.reshape(-1, 1, 28, 28).astype(np.float32) / 255

In [3]:
eps = 0.3

In [4]:
network1 = DeepConvNet()

network1.load_params("deep_convnet_params.pkl")
print("loaded Network Parameters!")

loaded Network Parameters!


In [5]:
network2 = DeepConvNet()

network2.load_params("deep_convnet_params_fgsm.pkl")
print("loaded Network Parameters!")

loaded Network Parameters!


In [6]:
def change_noise_list(x_list, eps):
    noise_list = []
    for x in x_list:
        random_noise1 = np.random.normal(loc = 0, scale = 1, size = (1, 28, 28))
        noise = (x + eps * np.sign(random_noise1)).clip(min=0, max=1)
        noise_list.append(noise)
        
    noise_list = np.array(noise_list)
    return noise_list

In [7]:
def generate_adv(x, label, network, eps=0.01):
    d, g = network.gradient_for_fgsm(x.reshape(1, 1, 28, 28), np.array([label]))
    d = d.reshape(28, 28)
    p = eps * np.sign(d)
    adv = (x + p).clip(min=0, max=1)
    adv = adv.reshape(1, 1, 28, 28)
    return adv

In [8]:
def change_adv_list(x_list, t_list, network, eps):
    adv_list = []
    for x, t in zip(x_list, t_list):
        adv = generate_adv(x, t, network, eps)
        adv_list.append(adv.reshape(1, 28, 28))
        
    adv_list = np.array(adv_list)
    return adv_list

In [9]:
noised_test = change_noise_list(x_test, eps)
noised_test.shape

(10000, 1, 28, 28)

In [10]:
fgsm_test = change_adv_list(x_test, t_test, network1, eps)
fgsm_test.shape

(10000, 1, 28, 28)

In [11]:
print("純粋なモデル")
print("無加工に対する正答率：", network1.accuracy(x_test, t_test))
print("ランダムノイズに対する正答率：", network1.accuracy(noised_test, t_test))
print("FGSMに対する正答率：", network1.accuracy(fgsm_test, t_test))

純粋なモデル
無加工に対する正答率： 0.9925
ランダムノイズに対する正答率： 0.8991
FGSMに対する正答率： 0.3703


In [12]:
print("FGSM学習モデル")
print("無加工に対する正答率：", network2.accuracy(x_test, t_test))
print("ランダムノイズに対する正答率：", network2.accuracy(noised_test, t_test))
print("純粋なモデルで作成したFGSMに対する正答率：", network2.accuracy(fgsm_test, t_test))

FGSM学習モデル
無加工に対する正答率： 0.981
ランダムノイズに対する正答率： 0.9669
純粋なモデルで作成したFGSMに対する正答率： 0.9831


In [13]:
fgsm_test_noise = change_adv_list(x_test, t_test, network2, eps)
fgsm_test_noise.shape

(10000, 1, 28, 28)

In [14]:
print("FGSM学習モデルで作成したFGSMに対する正答率：", network2.accuracy(fgsm_test_noise, t_test))

FGSM学習モデルで作成したFGSMに対する正答率： 0.3491
